# Dataset Extraction

This is a notebook for getting started with the data.

The dataset used is **SOLAQUA**, available from [SINTEF Open Data](https://data.sintef.no/feature/fe-a8f86232-5107-495e-a3dd-a86460eebef6).  


## Installing Packages

In [1]:
from collections import defaultdict 
from rosbags.highlevel import AnyReader
from pathlib import Path
import numpy as np
import cv2
import re

## Defining .bag files

All data files should be placed in the `../data` folder.

- `*_data.bag` → contains **sensor data** (ROS bag format).
- `*_video.bag` → contains **video images** (ROS bag format).

The dataset used is **SOLAQUA**, available from [SINTEF Open Data](https://data.sintef.no/feature/fe-a8f86232-5107-495e-a3dd-a86460eebef6).  

In [2]:
# Change these two lines to switch dataset
DATA_BAG  = Path("../data/2024-08-20_17-22-40_data.bag")   # sensor data
VIDEO_BAG = Path("../data/2024-08-20_17-22-40_video.bag")  # camera and sonar video

# Output folder for extracted frames, videos, sonar arrays, etc.
OUT_ROOT = Path("./output")
OUT_ROOT.mkdir(parents=True, exist_ok=True)

print(f"Using data bag : {DATA_BAG.resolve()}")
print(f"Using video bag: {VIDEO_BAG.resolve()}")
print(f"Output folder : {OUT_ROOT.resolve()}")

Using data bag : /Users/henrik/kode/SP/data/2024-08-20_17-22-40_data.bag
Using video bag: /Users/henrik/kode/SP/data/2024-08-20_17-22-40_video.bag
Output folder : /Users/henrik/kode/SP/notebooks/output


## List Topics

In [3]:
def human_hz(count, duration_s):
    if count == 0 or duration_s <= 0:
        return 0.0
    return count / duration_s

for bag in [DATA_BAG, VIDEO_BAG]:
    print(f"\n=== {bag.name} ===")
    if not bag.exists():
        print("  (missing)")
        continue

    counts = defaultdict(int)
    first_ts = defaultdict(lambda: None)
    last_ts  = defaultdict(lambda: None)
    types = {}

    with AnyReader([bag]) as r:
        for c in r.connections:
            types[c.topic] = c.msgtype
        for conn, ts, _ in r.messages():
            t = conn.topic
            counts[t] += 1
            if first_ts[t] is None or ts < first_ts[t]:
                first_ts[t] = ts
            if last_ts[t] is None or ts > last_ts[t]:
                last_ts[t] = ts

    if not counts:
        print("  (no messages)")
        continue

    col_topic = max(len(t) for t in counts.keys())
    col_type  = max(len(types.get(t, "")) for t in counts.keys())
    header = f"{'TOPIC'.ljust(col_topic)}  {'TYPE'.ljust(col_type)}  COUNT    START(ns)          END(ns)            DURATION(s)  ~HZ"
    print(header)
    print("-" * len(header))

    for t in sorted(counts.keys()):
        n = counts[t]
        t0 = first_ts[t]
        t1 = last_ts[t]
        dur_s = (t1 - t0) / 1e9 if (t0 is not None and t1 is not None) else 0.0
        hz = human_hz(n, dur_s)
        print(
            f"{t.ljust(col_topic)}  "
            f"{types.get(t,'').ljust(col_type)}  "
            f"{str(n).rjust(5)}    "
            f"{str(t0).rjust(16)}  "
            f"{str(t1).rjust(16)}  "
            f"{dur_s:11.3f}  {hz:5.2f}"
        )


=== 2024-08-20_17-22-40_data.bag ===
TOPIC                                     TYPE                                     COUNT    START(ns)          END(ns)            DURATION(s)  ~HZ
--------------------------------------------------------------------------------------------------------------------------------------------------
/bluerov2/alive                           std_msgs/msg/Float32                        99    1724167364244098600  1724167413076609100       48.833   2.03
/bluerov2/armed                           std_msgs/msg/Float32                        97    1724167365314919100  1724167413076555200       47.762   2.03
/bluerov2/battery                         messages/msg/BatteryStatus                  44    1724167367832934100  1724167412718299200       44.885   0.98
/bluerov2/modes                           joystick/msg/ModeManager2                    2    1724167365207571000  1724167410820708800       45.613   0.04
/commanded_thrust                         rospy_tutorial

## Extracting data

### Extract frames for _video.bag 

Remeber to handle if _video.bag has compressed and uncomressed images. 

In [4]:
# Extract camera frames from the selected VIDEO_BAG
# Layout:
#   ./output/<bag_timestamp>/camera/image_frames/camera_<ros_timestamp>.jpg

# Derive timestamp folder name from the bag (strip "_video" suffix)
bag_stem = VIDEO_BAG.stem.replace("_video", "")

# Build new structure: output/<timestamp>/camera/image_frames
RUN_ROOT = OUT_ROOT / bag_stem / "camera"
FRAMES_DIR = RUN_ROOT / "image_frames"
FRAMES_DIR.mkdir(parents=True, exist_ok=True)

assert VIDEO_BAG.exists(), f"Missing video bag: {VIDEO_BAG}"

saved = 0
skipped = 0

def save_frame(ts_ns: int, frame_bgr: np.ndarray):
    """Save a BGR frame as JPEG with timestamp-based filename."""
    out = FRAMES_DIR / f"camera_{ts_ns}.jpg"
    cv2.imwrite(str(out), frame_bgr)

print(f"[INFO] Reading {VIDEO_BAG.name}")
print(f"[INFO] Saving frames to: {FRAMES_DIR}")

with AnyReader([VIDEO_BAG]) as r:
    for i, (conn, ts, raw) in enumerate(r.messages()):
        msgtype = conn.msgtype

        # --- CompressedImage (JPEG/PNG) ---
        if msgtype == "sensor_msgs/msg/CompressedImage":
            msg = r.deserialize(raw, msgtype)
            arr = np.frombuffer(msg.data, np.uint8)
            frame = cv2.imdecode(arr, cv2.IMREAD_COLOR)
            if frame is None:
                skipped += 1
                continue
            save_frame(ts, frame)
            saved += 1

        # --- Raw Image (uncompressed) ---
        elif msgtype == "sensor_msgs/msg/Image":
            msg = r.deserialize(raw, msgtype)
            h, w = msg.height, msg.width
            step = msg.step
            buf = np.frombuffer(msg.data, dtype=np.uint8)

            chans = step // w if step % w == 0 else 3
            if chans == 1:
                frame = buf.reshape(h, step)[:, :w]
                frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
            else:
                frame = buf.reshape(h, step)[:, :w*chans].reshape(h, w, chans)
                if msg.encoding.lower() == "rgb8":
                    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

            save_frame(ts, frame)
            saved += 1

        # Progress prints
        if saved and (saved % 100 == 0):
            print(f"[INFO] Saved {saved} frames so far …")

print(f"[DONE] Saved {saved} frames to {FRAMES_DIR}")
if skipped:
    print(f"[WARN] Skipped {skipped} frames (decode failures).")


[INFO] Reading 2024-08-20_17-22-40_video.bag
[INFO] Saving frames to: output/2024-08-20_17-22-40/camera/image_frames
[INFO] Saved 100 frames so far …
[INFO] Saved 200 frames so far …
[INFO] Saved 300 frames so far …
[INFO] Saved 400 frames so far …
[INFO] Saved 400 frames so far …
[INFO] Saved 500 frames so far …
[INFO] Saved 500 frames so far …
[INFO] Saved 600 frames so far …
[INFO] Saved 600 frames so far …
[INFO] Saved 700 frames so far …
[INFO] Saved 700 frames so far …
[INFO] Saved 800 frames so far …
[INFO] Saved 800 frames so far …
[INFO] Saved 900 frames so far …
[INFO] Saved 900 frames so far …
[INFO] Saved 1000 frames so far …
[INFO] Saved 1000 frames so far …
[INFO] Saved 1100 frames so far …
[INFO] Saved 1200 frames so far …
[INFO] Saved 1200 frames so far …
[DONE] Saved 1275 frames to output/2024-08-20_17-22-40/camera/image_frames


### Make MP4 for _video.bag


In [5]:
# Build MP4 from frames in ./output/<timestamp>/camera/image_frames/
# Save as    ./output/<timestamp>/camera/<timestamp>.mp4

bag_stem   = VIDEO_BAG.stem.replace("_video", "")
RUN_ROOT   = OUT_ROOT / bag_stem / "camera"
FRAMES_DIR = RUN_ROOT / "image_frames"
OUT_MP4    = RUN_ROOT / f"{bag_stem}.mp4"

assert FRAMES_DIR.exists(), f"Frame folder not found: {FRAMES_DIR}. Extract frames first."

# Sort frames by ROS timestamp embedded in filename
ts_re = re.compile(r"camera_(\d+)\.jpg$")
def frame_key(p: Path):
    m = ts_re.search(p.name)
    return int(m.group(1)) if m else 0

frames = sorted(FRAMES_DIR.glob("camera_*.jpg"), key=frame_key)
assert frames, f"No frames found in {FRAMES_DIR}"

# Estimate capture FPS from timestamps (max possible)
ts_list = [frame_key(p) for p in frames if ts_re.search(p.name)]
if len(ts_list) > 1:
    dur_s = (ts_list[-1] - ts_list[0]) / 1e9
    fps_est = len(ts_list) / dur_s if dur_s > 0 else 15.0
else:
    fps_est = 15.0
FPS = round(fps_est, 2)

# Video dimensions from first frame
first = cv2.imread(str(frames[0]))
assert first is not None, f"Failed to read first frame: {frames[0]}"
h, w = first.shape[:2]

fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # use 'avc1' if your OpenCV supports H.264
vw = cv2.VideoWriter(str(OUT_MP4), fourcc, FPS, (w, h))
assert vw.isOpened(), "VideoWriter failed to open. Check codec availability."

print(f"[INFO] Writing {len(frames)} frames → {OUT_MP4}")
print(f"[INFO] FPS={FPS}  size={w}x{h}")

written = 0
for i, fp in enumerate(frames, 1):
    img = cv2.imread(str(fp))
    if img is None:
        continue
    if img.shape[:2] != (h, w):
        img = cv2.resize(img, (w, h), interpolation=cv2.INTER_AREA)
    vw.write(img)
    written += 1
    if written % 100 == 0:
        print(f"[INFO] Wrote {written}/{len(frames)} frames …")

vw.release()
print(f"[DONE] MP4 saved: {OUT_MP4}  ({written} frames at {FPS} FPS)")


[INFO] Writing 1275 frames → output/2024-08-20_17-22-40/camera/2024-08-20_17-22-40.mp4
[INFO] FPS=25.02  size=1280x720
[INFO] Wrote 100/1275 frames …
[INFO] Wrote 200/1275 frames …
[INFO] Wrote 300/1275 frames …
[INFO] Wrote 400/1275 frames …
[INFO] Wrote 500/1275 frames …
[INFO] Wrote 600/1275 frames …
[INFO] Wrote 700/1275 frames …
[INFO] Wrote 800/1275 frames …
[INFO] Wrote 900/1275 frames …
[INFO] Wrote 1000/1275 frames …
[INFO] Wrote 1100/1275 frames …
[INFO] Wrote 1200/1275 frames …
[DONE] MP4 saved: output/2024-08-20_17-22-40/camera/2024-08-20_17-22-40.mp4  (1275 frames at 25.02 FPS)


### Extract raw sonar frames to .npy 

In [6]:
# Extract raw SonoptixECHO pings from VIDEO_BAG to .npy (exact float32 arrays)
# Output:
#   ./output/<timestamp>/echo/raw_frames/sonar_<ros_ts>.npy



assert VIDEO_BAG.exists(), f"Missing video bag: {VIDEO_BAG}"

# Derive <timestamp> folder from VIDEO_BAG (strip "_video")
bag_stem = VIDEO_BAG.stem.replace("_video", "")

ECHO_ROOT = OUT_ROOT / bag_stem / "echo"
RAW_DIR   = ECHO_ROOT / "raw_frames"
RAW_DIR.mkdir(parents=True, exist_ok=True)

saved_raw = 0
skipped   = 0

print(f"[INFO] Reading {VIDEO_BAG.name}")
print(f"[INFO] RAW out: {RAW_DIR}")

with AnyReader([VIDEO_BAG]) as r:
    for i, (conn, ts, raw) in enumerate(r.messages()):
        if conn.msgtype != "sensors/msg/SonoptixECHO":
            continue

        msg  = r.deserialize(raw, conn.msgtype)
        data = np.asarray(msg.array_data.data, dtype=np.float32)

        # Determine H, W from layout (fallback heuristics)
        dims = msg.array_data.layout.dim
        H = int(dims[0].size) if len(dims) > 0 else 1024
        W = int(dims[1].size) if len(dims) > 1 else 256

        if data.size != H * W:
            if data.size == 1024 * 256:
                H, W = 1024, 256
            elif data.size == 256 * 1024:
                H, W = 256, 1024
            else:
                print(f"[WARN] size mismatch: vec={data.size}, layout={H}x{W}; skip ts={ts}")
                skipped += 1
                continue

        sonar_raw = data.reshape(H, W)
        np.save(RAW_DIR / f"sonar_{ts}.npy", sonar_raw)
        saved_raw += 1

        if saved_raw % 50 == 0:
            finite = np.isfinite(sonar_raw)
            if np.any(finite):
                mn = float(np.min(sonar_raw[finite]))
                mx = float(np.max(sonar_raw[finite]))
                print(f"[INFO] {saved_raw} raw saved … shape={H}x{W}, min..max={mn:.3g}..{mx:.3g}")
            else:
                print(f"[INFO] {saved_raw} raw saved … (no finite values)")

print(f"[DONE] RAW frames: {saved_raw}  → {RAW_DIR}")
if skipped:
    print(f"[WARN] Skipped frames: {skipped}")


[INFO] Reading 2024-08-20_17-22-40_video.bag
[INFO] RAW out: output/2024-08-20_17-22-40/echo/raw_frames
[INFO] 50 raw saved … shape=1024x256, min..max=0..64
[INFO] 100 raw saved … shape=1024x256, min..max=0..64
[INFO] 150 raw saved … shape=1024x256, min..max=0..64
[INFO] 200 raw saved … shape=1024x256, min..max=0..64
[INFO] 250 raw saved … shape=1024x256, min..max=0..64
[INFO] 300 raw saved … shape=1024x256, min..max=0..62
[INFO] 350 raw saved … shape=1024x256, min..max=0..64
[INFO] 400 raw saved … shape=1024x256, min..max=0..63
[INFO] 450 raw saved … shape=1024x256, min..max=0..63
[INFO] 500 raw saved … shape=1024x256, min..max=0..62
[INFO] 550 raw saved … shape=1024x256, min..max=0..62
[INFO] 600 raw saved … shape=1024x256, min..max=0..62
[INFO] 650 raw saved … shape=1024x256, min..max=0..63
[INFO] 700 raw saved … shape=1024x256, min..max=0..62
[INFO] 750 raw saved … shape=1024x256, min..max=0..63
[DONE] RAW frames: 795  → output/2024-08-20_17-22-40/echo/raw_frames


In [7]:
# Create preview PNGs from saved sonar .npy frames (visualization only)
# Input : ./output/<timestamp>/echo/raw_frames/sonar_<ros_ts>.npy
# Output: ./output/<timestamp>/echo/quicklook/sonar_<ros_ts>.png

# Use the same timestamp folder as above (from VIDEO_BAG)
bag_stem    = VIDEO_BAG.stem.replace("_video", "")
ECHO_ROOT   = OUT_ROOT / bag_stem / "echo"
RAW_DIR     = ECHO_ROOT / "raw_frames"
PREVIEW_DIR = ECHO_ROOT / "quicklook"
PREVIEW_DIR.mkdir(parents=True, exist_ok=True)

assert RAW_DIR.exists(), f"Raw frames folder not found: {RAW_DIR}"

# --- Preview normalization mode ---
# 'clip01': assumes data roughly in [0,1] and clips outside
# 'p01'   : per-frame 1st..99th percentile stretch (more legible)
PREVIEW_MODE = "p01"

def preview_u8(frame: np.ndarray) -> np.ndarray:
    v = frame.astype(np.float32)
    if PREVIEW_MODE == "clip01":
        v = np.clip(v, 0.0, 1.0)
        v = (v * 255.0).astype(np.uint8)
    else:
        finite = np.isfinite(v)
        if not np.any(finite):
            return np.zeros_like(v, dtype=np.uint8)
        lo, hi = np.percentile(v[finite], [1.0, 99.0])
        if hi <= lo:
            hi = lo + 1e-6
        v = np.clip((v - lo) / (hi - lo), 0.0, 1.0)
        v = (v * 255.0).astype(np.uint8)
    # Optional transpose for display
    return v.T

# Sort by ROS timestamp in filename
re_ts = re.compile(r"sonar_(\d+)\.npy$")
def npy_key(p: Path):
    m = re_ts.search(p.name)
    return int(m.group(1)) if m else 0

npy_files = sorted(RAW_DIR.glob("sonar_*.npy"), key=npy_key)
assert npy_files, f"No .npy frames found in {RAW_DIR}"

saved_png = 0
for i, npy_path in enumerate(npy_files, 1):
    arr = np.load(npy_path)
    png = preview_u8(arr)
    out = PREVIEW_DIR / (npy_path.stem + ".png")
    cv2.imwrite(str(out), png)
    saved_png += 1
    if saved_png % 50 == 0:
        print(f"[INFO] Wrote {saved_png} previews …")

print(f"[DONE] Quicklook PNGs: {saved_png} → {PREVIEW_DIR}")
print(f"[INFO] Preview mode: {PREVIEW_MODE}")


[INFO] Wrote 50 previews …
[INFO] Wrote 100 previews …
[INFO] Wrote 150 previews …
[INFO] Wrote 200 previews …
[INFO] Wrote 250 previews …
[INFO] Wrote 300 previews …
[INFO] Wrote 350 previews …
[INFO] Wrote 400 previews …
[INFO] Wrote 450 previews …
[INFO] Wrote 500 previews …
[INFO] Wrote 550 previews …
[INFO] Wrote 600 previews …
[INFO] Wrote 650 previews …
[INFO] Wrote 700 previews …
[INFO] Wrote 750 previews …
[DONE] Quicklook PNGs: 795 → output/2024-08-20_17-22-40/echo/quicklook
[INFO] Preview mode: p01
